In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

warnings.filterwarnings("ignore")

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
%run models.ipynb

In [27]:
dataset = "kbp37"
columns = ['sentence_length', 'root_word', 'root_index',
       'entity_1_index', 'entity_2_index', 'entity_distance',
       'no_words_before_entity_1', 'no_words_after_entity_2',
       'entity_1_root_distance', 'entity_2_root_distance', 'entity 1 name',
       'entity 2 name', 'entity_pos_1',
       'entity_pos_2', 'entity_dep_1', 'entity_dep_2', 'shortest_distance', 'et-label']

features = pd.read_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Latest/features_"+dataset+".csv", index_col=0)
features = features[columns]
features = features.rename(columns = {"et-label": "label"})
features.head(10)
# features['entity_type_1'] = features['entity 1 type'].fillna("OTHER")
# features['entity_type_2'] = features['entity 2 type'].fillna("OTHER")
# features = features.drop(["entity 1 type", "entity 2 type"], axis=1)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Latest/features_kbp37.csv'

## Feature Encoding

In [22]:
# ## For Entity Type Labels
# ENT_TYPES = ["PER", "GPE", "LOC", "ORG"]
# def recode_entity_types(value):
#     if value.lower() in ["person", "per"]:
#         return "PER"
#     elif value.lower() in ["organisation", "org"]:
#         return "ORG"
#     elif value.lower() in ["gpe"]:
#         return "GPE"
#     elif value.lower() in ["location", "loc"]:
#         return "LOC"
#     else:
#         return "OTHER"

In [4]:
# features["entity_type_1"] = features["entity_type_1"].apply(lambda x : recode_entity_types(x.split("-")[0]))
# features["entity_type_2"] = features["entity_type_2"].apply(lambda x : recode_entity_types(x.split("-")[0]))
# ## Filter rows with OTHER entity type

KeyError: 'entity_type_1'

In [ ]:
# features["label"] = features.apply(lambda x : f"{x['entity_type_1']}-{x['entity_type_2']}", axis=1)

In [ ]:
# print(f"Number of unique entity-type-paris : {len(features['label'].unique())}")
# print(f"Number of unique entity-type-paris : {features['label'].unique()}")

In [ ]:
# features.drop(["entity_type_1","entity_type_2"], axis=1, inplace=True)

In [ ]:
# features

## Multiclass Classification

In [24]:
features = features.groupby('label').filter(lambda x : len(x)>2)

In [25]:
features["label"].unique()

array(['PER-PER', 'PER-LOC', 'LOC-PER'], dtype=object)

In [26]:
# features_with_labels = pd.read_csv("features_"+dataset+".csv")
# features = features.groupby('label').apply(lambda x: x.sample(frac=0.2))

X = features.iloc[:,:-1]
X = X.fillna("")
y_original =  features.iloc[:,-1]

x_train_original, x_test_original, y_train_original, y_test_original = train_test_split(X, y_original, test_size=0.2, stratify=y_original)
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
x_train = onehot_encoder.fit_transform(x_train_original)
x_test = onehot_encoder.transform(x_test_original)

le = LabelEncoder()
y_train = le.fit_transform(y_train_original)
y_test = le.transform(y_test_original)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'LOC-PER': 0, 'PER-LOC': 1, 'PER-PER': 2}

In [89]:
unique, count = np.unique(y_original, return_counts = True)
dict(zip(unique, count))

{'LOC-LOC': 51,
 'ORG-LOC': 135,
 'ORG-ORG': 92,
 'PER-LOC': 29,
 'PER-ORG': 30,
 'PER-PER': 30}

In [90]:
pred = logistic_regression_multiple_classes(x_train, x_test, y_train, y_test, "et")
x_test_log_pred = x_test_original
x_test_log_pred["label"] = y_test
x_test_log_pred["prediction"] = pred
x_test_log_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_logisitic_regression_prediction.csv")

--------Logistic Regression-----------


In [91]:
pred = random_forest_multiple(x_train, x_test, y_train, y_test,"et")
x_test_rf_pred = x_test_original
x_test_rf_pred["label"] = y_test
x_test_rf_pred["prediction"] = pred
x_test_rf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_random_forest_prediction.csv")


--------Random Forest-----------


In [92]:
pred = svm_multiple("linear", x_train, x_test, y_train, y_test, "et")
x_test_svm_pred = x_test_original
x_test_svm_pred["label"] = y_test
x_test_svm_pred["prediction"] = pred
x_test_svm_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_prediction.csv")

--------Support Vector Classifier linear-----------


In [93]:
pred = svm_multiple("rbf", x_train, x_test, y_train, y_test, "et")
x_test_svmrbf_pred = x_test_original
x_test_svmrbf_pred["label"] = y_test
x_test_svmrbf_pred["prediction"] = pred
x_test_svmrbf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_rbf_prediction.csv")

--------Support Vector Classifier rbf-----------


In [94]:
pred = svm_multiple("poly", x_train, x_test, y_train, y_test, "et")
x_test_svmpoly_pred = x_test_original
x_test_svmpoly_pred["label"] = y_test
x_test_svmpoly_pred["prediction"] = pred
x_test_svmpoly_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_poly_prediction.csv")


--------Support Vector Classifier poly-----------


In [95]:
pred = xgboost_multiple_class(x_train, x_test, y_train, y_test, "et")
x_test_xg_pred = x_test_original
x_test_xg_pred["label"] = y_test
x_test_xg_pred["prediction"] = pred
x_test_xg_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_xgboost_prediction.csv")

In [96]:
pred = adaboost_multiple(x_train, x_test, y_train, y_test, "et")
x_test_ada_pred = x_test_original
x_test_ada_pred["label"] = y_test
x_test_ada_pred["prediction"] = pred
x_test_ada_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_adaboost_prediction.csv")
